In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import pandas as pd

2024-02-29 15:51:59.498829: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 15:51:59.498929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 15:51:59.659185: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
with open('/kaggle/input/english-and-sanscrit-words/fra-eng/fra.txt') as f:
    lines = f.read().split("\n")[:-1]
    print(len(lines))
text_pairs = []
for line in lines[:5000]:
    nep, eng, _ = line.split("\t")
    eng = 'START_ '+eng+' _END'
    text_pairs.append((nep, eng))

229803


In [3]:
df = pd.DataFrame(data=text_pairs, columns=['NEP', 'ENG'])

In [4]:
df

,NEP,ENG
0,Go.,START_ Va ! _END
1,Go.,START_ Marche. _END
2,Go.,START_ En route ! _END
3,Go.,START_ Bouge ! _END
4,Hi.,START_ Salut ! _END
...,...,...
4995,He denied it.,START_ Il l'a refusé. _END
4996,He denied it.,START_ Il le refusa. _END
4997,He dozed off.,START_ Il s'est endormi. _END
4998,He dumped me.,START_ Il m'a plaquée. _END


In [5]:
# import nltk
# from nltk.corpus import stopwords
# nltk.download('stopwords')

# def remove_stopwords(text):
#     stop_words = set(stopwords.words("english"))
#     words = text.split()
#     filtered_text = [word for word in words if word.lower() not in stop_words]
#     return ' '.join(filtered_text)

In [6]:
# df.ENG = df.ENG.apply(remove_stopwords)

In [7]:
# Vocabulary of Nepali 
all_nepali_words=set()
for nep in df.NEP:
    for word in nep.split():
        if word not in all_nepali_words:
            all_nepali_words.add(word)
# Vocabulary of English
all_eng_words=set()
for eng in df.ENG:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

In [8]:
len(all_nepali_words), len(all_eng_words)

(1561, 3241)

In [9]:
# Max Length of source sequence
lenght_list=[]
for l in df.NEP:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

4

In [10]:
# Max Length of target sequence
lenght_list=[]
for l in df.ENG:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

12

In [11]:
input_words = sorted(list(all_nepali_words))
target_words = sorted(list(all_eng_words))
num_encoder_tokens = len(all_nepali_words)
num_decoder_tokens = len(all_eng_words)
num_encoder_tokens, num_decoder_tokens

(1561, 3241)

In [12]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

In [13]:
# padding texts
encoder_input_data = np.zeros(
    (len(df.NEP), max_length_src),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(df.ENG), max_length_tar),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(df.ENG), max_length_tar, num_decoder_tokens),
    dtype='float32')

In [14]:
for i, (input_text, target_text) in enumerate(zip(df.NEP, df.ENG)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [15]:
embedding_size = 50

In [16]:
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [17]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, embedding_size)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(50, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [18]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 50)  │     78,050 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 50)  │    162,050 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 50),      │     20,200 │ embedding[0][0]   │
│                     │ (None, 50),       │            │                   │
│                     │ (None, 50)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │     20,200 │ embedding_1[0][0… │
│                     │ 50), (None, 50),  │            │ lstm[0][1],       │
│                     │ (None, 50)]       │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None,      │    165,291 │ lstm_1[0][0]      │
│                     │ 3241)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 445,791 (1.70 MB)

 Trainable params: 445,791 (1.70 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=32,
          epochs=20,
          validation_split=0.05)

Epoch 1/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - acc: 0.0758 - loss: 2.3280 - val_acc: 0.0833 - val_loss: 2.1431
Epoch 2/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - acc: 0.0854 - loss: 1.7840 - val_acc: 0.0837 - val_loss: 2.1378
Epoch 3/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - acc: 0.0976 - loss: 1.7447 - val_acc: 0.0837 - val_loss: 2.1450
Epoch 4/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - acc: 0.0962 - loss: 1.7248 - val_acc: 0.0837 - val_loss: 2.1457
Epoch 5/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - acc: 0.0967 - loss: 1.7161 - val_acc: 0.0837 - val_loss: 2.1450
Epoch 6/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - acc: 0.1002 - loss: 1.6838 - val_acc: 0.0837 - val_loss: 2.1470
Epoch 7/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - acc: 0.1033 - loss: 1.6791 - val_acc: 0.0837 - val_loss: 2.1461
Epoch 8/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - acc: 0.1030 - loss: 1.6703 - val_acc: 0.0837 - val_loss: 2.1461
Epoch 9/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - a

In [20]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 50)       │        78,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ [(None, 50), (None,    │        20,200 │
│                                 │ 50), (None, 50)]       │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 98,250 (383.79 KB)

 Trainable params: 98,250 (383.79 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [22]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [23]:
for seq_index in [500, 450, 3000]:#[14077,20122,40035,40064, 40056, 40068, 40090, 40095, 40100, 40119, 40131, 40136, 40150, 40153]:
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', df.NEP[seq_index: seq_index + 1])
    print('Expected output:', df.ENG[seq_index:seq_index +1])
    print('Decoded sentence:', decoded_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
-
Input sentence: 500    Can I go?
Name: NEP, dtype: object
Expected output: 500    START_ Je peux y aller ? _END
Name: ENG, dtype: object
Decoded sentence:  Je suis _END
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
-
Input sentence: 450    Wash up.
Name: NEP, dtype: object
Expected output: 450    START_ Lavez-vous ! _END
Name: ENG, dtype: object
Decoded sentence:  Merci _END
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
-
Input sentence: 3000    Do it again!
Name: NEP, dtype: object
Expected output: 3000    START_ Fais-le de nouveau ! _END
Name: ENG, dtype: object
Decoded sentence:  À _END
